# How good will it go when we combine both methods together? 

Process: 
    - Generate text as usual with the grammar finetuned GPT model
    - Take the generated sample, split it on EOS tokens and grammar translate it with the translation model
    - Join it again. Now we got corrected Text that has much less errors in comparison to the standard GPT model
    - Outperforms the errorRate of the single models!

In [45]:
import pickle
from utility import *

## Step1 : Check how much errors the classic model makes

In [29]:
data = open("classic/gpt2_generationgenerate_cf_1.txt","r+")

In [30]:
data = data.read()
data = data.split("<|endoftext|>")
texts = data

In [31]:
prep = []
for block in texts:
    sentence = block.replace(". ", ".<|splitter|>")
    sentence = sentence.replace("? ", "?<|splitter|>")
    sentence = sentence.replace("! ", "!<|splitter|>")
    sentence = sentence.replace(".\n", ".\n<|splitter|>")
    sentence = sentence.replace(".\n\n", ".\n\n<|splitter|>")
    sentence = sentence.replace("?\n", "?\n<|splitter|>")
    sentence = sentence.replace("?\n\n", "?\n\n<|splitter|>")
    sentence = sentence.replace("!\n", "!\n<|splitter|>")
    sentence = sentence.replace("!\n\n", "!\n\n<|splitter|>")
    prep.append(sentence)
splitted = []
for block in prep: 
    splitted.append(block.split("<|splitter|>"))
final = [item for sublist in splitted for item in sublist]

In [37]:
%%capture
corrected,stats  = correct(final)

In [38]:
grammar_stats(stats)

[594, 1187, 55986, 0.02120172900367949, 1.187, 0]

## Step2 : build a translation dataset from the finetuned samples

In [33]:
condition = "The Technology Report empowers or enlightens. ==== The Technology Report empowers or enlightens."

In [34]:
examples = []
for x in final: 
    examples.append( condition + " " + x + "====" + x + "<|endoftext|>")

In [35]:
examples = " ".join(examples)

In [ ]:
examples[:1000]

In [ ]:
f = open("EOS_combined.txt", "w",encoding = "UTF-8")
f.write(examples)
f.close()

## Step3: translate them with our translation model

    - Finetuned 1 Epoch + 12 layer translation model
    - Finetuned 1 Epoch + 6layer translation model

In [43]:
a = pickle.load(open("Combine/combined.p","rb"))

In [46]:
%%capture
corrected, stats = correct(a)

In [47]:
grammar_stats(stats)

[447, 816, 55797, 0.014624442174310446, 0.816, 0]

In [48]:
b = pickle.load(open("Combine/combined_6.p","rb"))

In [51]:
%%capture
corrected, stats = correct(b)

In [52]:
grammar_stats(stats)

[480, 862, 55811, 0.015444983963734747, 0.862, 0]

## Step4: Be happy because it works  =) 